## Implementing Batch GD with Early Stopping for Softmax Regression by Hand

In [93]:
import numpy as np
from sklearn import datasets
from sklearn.linear_model import LogisticRegression

np.random.seed(2042)

iris = datasets.load_iris()
features = (2, 3)
classlen = len(iris["target_names"])
X = iris["data"][:, features]
y = iris["target"]
X_w_bias = np.c_[np.ones([len(X), 1]), X]

In [94]:
def normalizer(vec):
    return np.sum(np.exp(vec))

### Move around and separate data to training, testing, and validating

In [95]:
test_ratio = 0.2
validation_ratio = 0.2
total_size = len(X_w_bias)

# Determine how data should be distributed
test_size = int(total_size*test_ratio)
validation_size = int(total_size*validation_ratio)
train_size = total_size - test_size - validation_size

rnd_indices = np.random.permutation(total_size) #randomly rearrange training data

# Split data by its ratios
X_train = X_w_bias[rnd_indices[:train_size]]
y_train = y[rnd_indices[:train_size]]
X_test = X_w_bias[rnd_indices[train_size:-test_size]] #exclude test_size amt of data starting from the end
y_test = y[rnd_indices[train_size:-test_size]]
X_valid = X_w_bias[rnd_indices[-test_size:]]
y_valid = y[rnd_indices[-test_size:]]

### One-hot boys for the easy life

In [96]:
# create one-hot vector out of existing vector
def to_one_hot(vec):
    n_classes = vec.max() + 1 # max value in vector is 2, meaning 3 classes (since it was originally 0-indexed)
    m = len(vec)
    Y_one_hot = np.zeros((m, n_classes))
    Y_one_hot[np.arange(m), vec] = 1 
    # np.arange(m) returns evenly spaced values b/w 0 to m, so it will just output 0,1,...,m
    # y is the index of the correct class, so at Y_one_hot[i, correct_class_Ind] it will be 1
    return Y_one_hot

y_train[:10]

array([0, 1, 2, 1, 1, 0, 1, 1, 1, 0])

In [97]:
to_one_hot(y_train[:10])

array([[1., 0., 0.],
       [0., 1., 0.],
       [0., 0., 1.],
       [0., 1., 0.],
       [0., 1., 0.],
       [1., 0., 0.],
       [0., 1., 0.],
       [0., 1., 0.],
       [0., 1., 0.],
       [1., 0., 0.]])

In [98]:
y_train_one_hot = to_one_hot(y_train)
y_test_one_hot = to_one_hot(y_test)
y_valid_one_hot = to_one_hot(y_valid)

def softmax(vec):
    return np.exp(vec) / np.sum(np.exp(vec), axis=1, keepdims=True)

In [99]:
n_inputs = X_train.shape[1] # == 3 b/c 2 inputs and bias term
n_outputs = classlen

### Train a plain softmax model

In [100]:
eta = 0.01
n_iterations = 5001
m = len(X_train)
epsilon = 1e-7 #in case gradient hits x = 0 is not differentiable, we don't want errors

theta = np.random.rand(n_inputs, n_outputs) # random theta weights matrix. [(features + bias) x classes]

for it in range(n_iterations):
    logits = X_train.dot(theta) # [insts x (features + bias)] * [(features + bias) x class_probs] = [insts x class_probs]
    Y_proba = softmax(logits) # turn into probabilities per instance
    loss = -np.mean(np.sum(y_train_one_hot*np.log(Y_proba + epsilon), axis=1)) # cross entropy loss function
    error = Y_proba - y_train_one_hot
    if it % 1000 == 0:
        print(it, loss)
    grad = 1/m * X_train.T.dot(error) # 1/m * (X^T * (P.k - Y.k))
    # X_train.T = [(features + bias) x inst], error = [inst x class_probs]
    theta -= eta*grad

    # Won't get much lower than 0.39, if you print error instead you will see how small the error is

0 2.103801941146351
1000 0.6619775377544007
2000 0.5284247416017703
3000 0.46205628837815627
4000 0.4202882767705059
5000 0.3903164138905715


In [101]:
theta # trained weights

array([[ 3.52086430e+00, -1.81208371e-01, -2.48681789e+00],
       [ 1.00928135e-01,  1.08123288e+00,  9.24694983e-01],
       [-1.38699073e+00, -3.46989769e-03,  1.97108268e+00]])

In [102]:
logits = X_valid.dot(theta)
y_proba = softmax(logits)
y_predict = np.argmax(y_proba, axis=1) # get the index of the "1" value in the one-hot subvectors in the matrix

accuracy_score = np.mean(y_valid == y_predict) # takes the mean of a boolean vector (0 = wrong, 1 = correct)
accuracy_score

0.9

### Train a softmax model with some regularization

In [103]:
eta = 0.01
n_iterations = 5001
m = len(X_train)
epsilon = 1e-7 # in case gradient hits x = 0 is not differentiable, we don't want errors
alpha = 0.1 # regularization hyperparameter

theta = np.random.rand(n_inputs, n_outputs) # random theta weights matrix. [(features + bias) x classes]

for it in range(n_iterations):
    logits = X_train.dot(theta) # [insts x (features + bias)] * [(features + bias) x class_probs] = [insts x class_probs]
    Y_proba = softmax(logits) # turn into probabilities per instance
    cross_entropy_loss = -np.mean(np.sum(y_train_one_hot*np.log(Y_proba + epsilon), axis=1)) # cross entropy loss function
    ridge_loss = 1/2 * np.sum(theta[1:] ** 2) # sums the squares of all weights in the input arr
    
    # ( if it had been specified axis=1, then it would've summed along the rows. similarly, axis=2 sums along columns)
    
    loss = cross_entropy_loss + alpha * ridge_loss
    error = Y_proba - y_train_one_hot
    if it % 1000 == 0:
        print(it, loss)
    grad = 1/m * X_train.T.dot(error) + np.r_[np.zeros([1, n_outputs]), alpha * theta[1:]] 
    # 1/m * (X^T * (P.k - Y.k)) + regularization
    # X_train.T = [(features + bias) x inst], error = [inst x class_probs]
    # bias gradients are 0, appended as column 0 for all instances
    theta -= eta*grad

    # Loss seems greater than before, but perhaps it will do better w/ less overfitting

0 1.0432300272914143
1000 0.6858662577216126
2000 0.598785478354788
3000 0.562467148871917
4000 0.5425341016720427
5000 0.5298213143367644


In [104]:
logits = X_valid.dot(theta)
y_proba = softmax(logits)
y_predict = np.argmax(y_proba, axis=1)

accuracy_score = np.mean(y_predict == y_valid)
accuracy_score

0.9333333333333333

### Regularization + Early Stopping Softmax Model

In [115]:
eta = 0.01
n_iterations = 10000
m = len(X_train)
epsilon = 1e-7 # in case gradient hits x = 0 is not differentiable, we don't want errors
alpha = 0.1 # regularization hyperparameter
overfitLimit = 5
overfitCount = 0 # allow overfitLimit amt of times for error to increase before stopping model
minLoss = 0 # track previous loss amt

theta = np.random.rand(n_inputs, n_outputs) # random theta weights matrix. [(features + bias) x classes]

for it in range(n_iterations):
    logits = X_train.dot(theta) # [insts x (features + bias)] * [(features + bias) x class_probs] = [insts x class_probs]
    Y_proba = softmax(logits) # turn into probabilities per instance
    cross_entropy_loss = -np.mean(np.sum(y_train_one_hot*np.log(Y_proba + epsilon), axis=1)) # cross entropy loss function
    ridge_loss = 1/2 * np.sum(theta[1:] ** 2) # sums the squares of all weights in the input arr
    
    # ( if it had been specified axis=1, then it would've summed along the rows. similarly, axis=2 sums along columns)

    loss = cross_entropy_loss + alpha * ridge_loss
    error = Y_proba - y_train_one_hot
    if it % 1000 == 0:
        print(it, loss)
    
    if it > 1 and minLoss < loss:
        print("final iteration:", str(it))
        break
    else:
        minLoss = loss
        overfitCount = 0
    
    grad = 1/m * X_train.T.dot(error) + np.r_[np.zeros([1, n_outputs]), alpha * theta[1:]] 
    # 1/m * (X^T * (P.k - Y.k)) + regularization
    # X_train.T = [(features + bias) x inst], error = [inst x class_probs]
    # bias gradients are 0, appended as column 0 for all instances
    theta -= eta*grad

    # For some reason error doesn't go up = no overfitting?? However, this early stopping should conceptually work...

0 1.7794207116643639
1000 0.7006165328110734
2000 0.6027401729308076
3000 0.5639843447895967
4000 0.5432236134258799
5000 0.5301718706474263
6000 0.5211868720693656
7000 0.5146426003445033
8000 0.5096957217453889
9000 0.5058599537861246


In [116]:
logits = X_valid.dot(theta)
y_proba = softmax(logits)
y_predict = np.argmax(y_proba, axis=1)

accuracy_score = np.mean(y_predict == y_valid)
accuracy_score

0.9333333333333333